# 0. Install and Import dependencies

We will use the most advanced environment for RL-trading task - DI-engine/gym-anytrade. It's a part of DI-engine RL framework, which contains a lot of environments inside.

In [ ]:
!git clone https://github.com/opendilab/DI-engine.git
!pip install DI-engine/.

Cloning into 'DI-engine'...
remote: Enumerating objects: 27485, done.
remote: Counting objects: 100% (1682/1682), done.
remote: Compressing objects: 100% (702/702), done.
remote: Total 27485 (delta 1108), reused 1404 (delta 969), pack-reused 25803
Receiving objects: 100% (27485/27485), 232.21 MiB | 26.74 MiB/s, done.
Resolving deltas: 100% (19204/19204), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./DI-engine
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for DI-engine: filename=DI_engine-0.4.4-py3-none-any.whl size=1836395 sha256=e24e344c0db5367cd7e680d56d

Installation Check

In [ ]:
import ding
print(ding.__version__)

v0.4.4
